In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
drive_PATH = '../content/drive/MyDrive/Colab Notebooks/l101.experiments.1'

Mounted at /content/drive


In [2]:
!pip install jsonlines

In [3]:
from transformers import AutoTokenizer, RemBertModel
import torch

tokenizer = AutoTokenizer.from_pretrained("google/rembert")
model = RemBertModel.from_pretrained("google/rembert")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/263 [00:00<?, ?B/s]

sentencepiece.model:   0%|          | 0.00/4.70M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.71M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/686 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.30G [00:00<?, ?B/s]

In [4]:
def encode(sent):
    inputs = tokenizer(sent, return_tensors="pt")
    outputs = model(**inputs)
    last_hidden_states = outputs.last_hidden_state
    return last_hidden_states.detach().numpy()[0]

def encode_sentence(sent):
    enc = encode(sent)
    pooled_enc = sum(enc) / enc.shape[0]
    return pooled_enc

def encode_candidates(cands):
    return [encode_sentence(cand) for cand in cands]

In [5]:
import jsonlines
# BLEURT

bleurt_TREN_PATH = drive_PATH + '/data-main/bleurt-TREN.jsonl'
bleurt_TREN = []
with jsonlines.open(bleurt_TREN_PATH) as f:
    for line in f.iter():
        bleurt_TREN.append(line)
# chrF++
chrf_TREN_PATH = drive_PATH + '/data-main/chrf-TREN.jsonl'
chrf_TREN = []
with jsonlines.open(chrf_TREN_PATH) as f:
    for line in f.iter():
        chrf_TREN.append(line)

In [6]:
num_examples = 1000
assert(len(chrf_TREN) == num_examples)
assert(len(bleurt_TREN) == num_examples)

for i in range(num_examples):
  assert(chrf_TREN[i]['src'] == bleurt_TREN[i]['src'])
  assert(chrf_TREN[i]['ref'] == bleurt_TREN[i]['ref'])
  assert(chrf_TREN[i]['candidates'] == bleurt_TREN[i]['candidates'])


In [7]:
import tqdm

TREN = []
for i in tqdm.tqdm(range(num_examples)):
  TREN_dict = { 'src': chrf_TREN[i]['src'], 'ref': chrf_TREN[i]['ref'], 'candidates': chrf_TREN[i]['candidates'], 'enc_candidates':encode_candidates(chrf_TREN[i]['candidates']) }
  ### bleurt
  # cand scores
  TREN_dict['BLEURT_cand_scores'] = bleurt_TREN[i]['BLEURT_cand_scores']
  # mbr cand scores
  TREN_dict['MBR_BLEURT_cand_scores'] = bleurt_TREN[i]['MBR_BLEURT_cand_scores']
  # mbr score matrix
  TREN_dict['MBR_BLEURT_score_matrix'] = bleurt_TREN[i]['MBR_BLEURT_score_matrix']
  # mbr winner index
  TREN_dict['MBR_BLEURT_winner_index'] = bleurt_TREN[i]['MBR_BLEURT_winner_index']
  # mbr winner
  TREN_dict['MBR_BLEURT_winner'] = bleurt_TREN[i]['MBR_BLEURT_winner']
  ### chrf
  # cand scores
  TREN_dict['CHRF_cand_scores'] = chrf_TREN[i]['CHRF_cand_scores']
  # mbr cand scores
  TREN_dict['MBR_CHRF_cand_scores'] = chrf_TREN[i]['MBR_CHRF_cand_scores']
  # mbr score matrix
  TREN_dict['MBR_CHRF_score_matrix'] = chrf_TREN[i]['MBR_CHRF_score_matrix']
  # mbr winner index
  TREN_dict['MBR_CHRF_winner_index'] = chrf_TREN[i]['MBR_CHRF_winner_index']
  # mbr winner
  TREN_dict['MBR_CHRF_winner'] = chrf_TREN[i]['MBR_CHRF_winner']
  ###
  TREN.append(TREN_dict)

100%|██████████| 1000/1000 [9:01:55<00:00, 32.52s/it]


In [8]:
import numpy as np
for i in tqdm.tqdm(range(num_examples)):
  TREN[i]['enc_candidates'] = np.array(TREN[i]['enc_candidates'], dtype=float).tolist()

100%|██████████| 1000/1000 [00:06<00:00, 148.13it/s]


In [9]:
TREN_PATH = drive_PATH+ f'/data-main/TREN.jsonl'
with jsonlines.open(TREN_PATH, mode='w') as writer:
    for item in TREN:
        writer.write(item)